In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import CollisionModule

In [ ]:
import cProfile

In [ ]:
import torch

In [ ]:
release_pos = np.load('./release_pos_1e8_1105.npy')
velosity_matrix = np.load('./velosity_matrix_1e8_1105.npy')
DXsec_Al = np.load('./DXsec_Al_1e8.npy')

In [ ]:
print(release_pos.shape)

fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (edge)")
ax.hist(release_pos[:,0], bins=100,histtype='step', stacked=True, fill=False, label='pos_x')
ax.hist(release_pos[:,1], bins=100,histtype='step', stacked=True, fill=False, label='pos_y')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
print(velosity_matrix.shape)

fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (edge)")
ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
TS = 0.14 #200mm
release_pos[:,2] += TS
print(release_pos[:,0].min())
print(release_pos[:,0].max())
print(release_pos[:,1].min())
print(release_pos[:,1].max())

In [ ]:
logname = 'DSMC_0708_TS180_collision_ArAl_SMD'
transport = CollisionModule.transport(boundaryType='SMD', maxMove=1, pressure_pa=0.2, timeStep= 1e-7, temperature=300, cellSize=[230, 100, 100], celllength=0.002, chamberSize=[0.23, 0.1], DXsec=DXsec_Al, logname=logname)
transport.setXsec(energy_range=[0.1, 50, 50])

In [ ]:
cProfile.run('result = transport.runE(p0=release_pos, v0=velosity_matrix, time=1e-5)')

In [ ]:
result = transport.runE(p0=release_pos, v0=velosity_matrix, time=5e-4)

In [ ]:
result2 = np.load('./transport_TS300_result_posvel.npy')
result1 = np.load('./transport_TS300_result_collisionPos.npy')

In [ ]:
depo_count = 0
for i in range(len(result[1])):
    depo_count += result[1][i].shape[0]

depo_arr = np.zeros((depo_count, 6))
pointer = 0
for i in range(len(result[1])):
    depo_arr[pointer:pointer + result[1][i].shape[0]] = result[1][i]
    pointer += result[1][i].shape[0]


In [ ]:
collison_count = 0
for i in range(len(result[0])):
    collison_count += result[0][i].shape[0]

collison_arr = np.zeros((collison_count, 3))
pointer = 0
for i in range(len(result[0])):
    collison_arr[pointer:pointer + result[0][i].shape[0]] = result[0][i]
    pointer += result[0][i].shape[0]

In [ ]:
print(depo_arr.shape)
print(collison_arr.shape)

In [ ]:
print(depo_arr.shape)
print(collison_arr.shape)
np.save('transport_TS60_result_posvel_02Pa_SMD_final1104', depo_arr)
np.save('transport_TS60_result_collisionPos_02Pa_SMD_final1104', collison_arr)

In [ ]:
x_erosion_np = collison_arr[:,0]
y_erosion_np = collison_arr[:,1]

xbins = np.linspace(-0.2,0.2,200)
ybins = np.linspace(-0.1,0.1,100)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
y_bins = np.histogram(y_erosion_np, bins=ybins)

H, xedges, yedges = np.histogram2d(x_erosion_np, y_erosion_np, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('Ion position, erosion area(Top view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
x_erosion_np = collison_arr[:,0]
z_erosion_np = collison_arr[:,2]

xbins = np.linspace(-0.2,0.2,200)
zbins = np.linspace(0,0.2,100)
# zbins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
z_bins = np.histogram(z_erosion_np, bins=zbins)

H, xedges, zedges = np.histogram2d(x_erosion_np, z_erosion_np, bins=(xbins, zbins))

fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]])
ax.set_title('Al-Ar collision position(side view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
# fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
depo_pv = depo_arr
a = np.array(depo_pv)
print(a.shape)

In [ ]:
velosity_matrix.shape

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(depo_arr[:,3], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
# ax.hist(result[2][:,4], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
# ax.hist(result[2][:,5], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')

ax.hist(velosity_matrix[:,0], bins=100,histtype='step', stacked=True, fill=False, label='vel_x_2')
# ax.hist(velosity_matrix[:,1], bins=100,histtype='step', stacked=True, fill=False, label='vel_y_2')
# ax.hist(velosity_matrix[:,2], bins=100,histtype='step', stacked=True, fill=False, label='vel_z_2')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(a[:,3], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(a[:,4], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(a[:,5], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(np.linalg.norm(a[:,3:],axis=1), bins=100,histtype='step', stacked=True, fill=False, label='vel_x')

# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
x_erosion_np = a[:,0]
y_erosion_np = a[:,1]

xbins = np.linspace(-0.3,0.3,300)
ybins = np.linspace(-0.1,0.1,100)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
y_bins = np.histogram(y_erosion_np, bins=ybins)

H, xedges, yedges = np.histogram2d(x_erosion_np, y_erosion_np, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('deposition position(Top view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
# fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
center_indices = np.logical_or(a[:,0] < -0.015, a[:,0] > 0.015)
center_indices |= np.logical_or(a[:,1] < -0.015, a[:,1] > 0.015)
center_indices |= np.array(a[:,2] > 0.1)

if np.any(center_indices):
    center_depo = a[~center_indices]

print(center_depo.shape)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (center)")
ax.hist(center_depo[:,3], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(center_depo[:,4], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(center_depo[:,5], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
x_erosion_np = center_depo[:,0]
y_erosion_np = center_depo[:,1]

xbins = np.linspace(-0.3,0.3,300)
ybins = np.linspace(-0.1,0.1,100)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
y_bins = np.histogram(y_erosion_np, bins=ybins)

H, xedges, yedges = np.histogram2d(x_erosion_np, y_erosion_np, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('Ion position, erosion area(Top view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
edge_indices = np.logical_or(a[:,0] < 0.165, a[:,0] > 0.195)
edge_indices |= np.logical_or(a[:,1] < -0.015, a[:,1] > 0.015)
edge_indices |= np.array(a[:,2] > 0.2)
if np.any(edge_indices):
    edge_depo = a[~edge_indices]

print(edge_depo.shape)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (edge)")
ax.hist(edge_depo[:,3], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(edge_depo[:,4], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(edge_depo[:,5], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
x_erosion_np = edge_depo[:,0]
y_erosion_np = edge_depo[:,1]

xbins = np.linspace(-0.3,0.3,300)
ybins = np.linspace(-0.1,0.1,100)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
y_bins = np.histogram(y_erosion_np, bins=ybins)

H, xedges, yedges = np.histogram2d(x_erosion_np, y_erosion_np, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('Ion position, erosion area(Top view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
for i in range(1):
    center_depo = np.concatenate((center_depo, center_depo), axis=0)

print(center_depo.shape)

In [ ]:
import depoSimulator_symmetry

In [ ]:
film = np.zeros((88, 88, 120))

bottom = 10
film[:, :, 0:bottom] = 10 # bottom

height = 30
left_side = 24
right_side = 24
film[:, 88-right_side:, 0:height] = 10
film[:, 0:left_side, 0:height] = 10
film[88-right_side:, :, 0:height] = 10
film[0:left_side, :, 0:height] = 10

In [ ]:
substrute = torch.Tensor(film==10).to_sparse()
substrute = substrute.indices().numpy().T
# point_cloud = pv.PolyData(points)
# point_cloud.plot(eye_dome_lighting=True)

p = pv.Plotter()
# p.add_mesh(depo3, color='yellow', point_size=5)
p.add_mesh(substrute, color='lightblue', point_size=9)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
print(center_depo[:, 3:].shape)

In [ ]:
logname = 'TS300'
test = depoSimulator_symmetry.depo(param = [1.6, -0.7], TS = 0.3, N = center_depo.shape[0], sub_xy=[0,0], film=film, n=1, cellSize=[88, 88, 120], celllength=1, kdtreeN=5, tstep=1e-4, logname=logname)
deposit = test.run_afterCollision(1, 125, center_depo[:, 3:], 0.5)

In [ ]:
np.save('./TScompare/TS300_afterCollision_0530_v2_run1_t1e-5_p4e6', deposit)

In [ ]:
depo1 = torch.Tensor(deposit[20:46, :, :]==20).to_sparse()
depo1 = depo1.indices().numpy().T

# depo2 = torch.Tensor(deposit[:100, :, :]==40).to_sparse()
# depo2 = depo2.indices().numpy().T

# depo3 = torch.Tensor(np.logical_or( deposit[:100, :, :]==60, deposit[:100, :, :]==80)).to_sparse()
# depo3 = depo3.indices().numpy().T

substrute = torch.Tensor(deposit[20:46, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
# point_cloud = pv.PolyData(points)
# point_cloud.plot(eye_dome_lighting=True)

p = pv.Plotter() 
p.add_mesh(depo1, color='red', point_size=10)
# p.add_mesh(depo2, color='green', point_size=5)
# p.add_mesh(depo3, color='yellow', point_size=5)
p.add_mesh(substrute, color='lightblue', point_size=10)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(deposit[:, :, :]==20).to_sparse()
depo1 = depo1.indices().numpy().T

# depo2 = torch.Tensor(deposit[:100, :, :]==40).to_sparse()
# depo2 = depo2.indices().numpy().T

# depo3 = torch.Tensor(np.logical_or( deposit[:100, :, :]==60, deposit[:100, :, :]==80)).to_sparse()
# depo3 = depo3.indices().numpy().T

substrute = torch.Tensor(deposit[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
# point_cloud = pv.PolyData(points)
# point_cloud.plot(eye_dome_lighting=True)

p = pv.Plotter() 
p.add_mesh(depo1, color='red', point_size=10)
# p.add_mesh(depo2, color='green', point_size=5)
# p.add_mesh(depo3, color='yellow', point_size=5)
p.add_mesh(substrute, color='lightblue', point_size=10)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("velosity distribution of deposited aluminum on wafer (edge)")
ax.hist(edge_depo[:,3], bins=100,histtype='step', stacked=True, fill=False, label='vel_x')
ax.hist(edge_depo[:,4], bins=100,histtype='step', stacked=True, fill=False, label='vel_y')
ax.hist(edge_depo[:,5], bins=100,histtype='step', stacked=True, fill=False, label='vel_z')
# ax.set_yscale('log')
# ax.plot(X, hist_dist.cdf(X), label='CDF')
ax.legend()

In [ ]:
for i in range(3):
    edge_depo = np.concatenate((edge_depo, edge_depo), axis=0)

print(edge_depo.shape)

In [ ]:
logname = 'TS300'
test = depoSimulator_symmetry.depo(param = [1.6, -0.7], TS = 0.3, N = edge_depo.shape[0], sub_xy=[0,0], film=film, n=1, cellSize=[88, 88, 120], celllength=1, kdtreeN=5, tstep=1e-4, logname=logname)
deposit = test.run_afterCollision(1, 125, edge_depo[:, 3:], 0.5)

In [ ]:
np.save('./TScompare/TS300_afterCollision_0530_v2_run1_t1e-5_p4e6_edge', deposit)

In [ ]:
depo1 = torch.Tensor(deposit[20:46, 20:68, :]==20).to_sparse()
depo1 = depo1.indices().numpy().T

# depo2 = torch.Tensor(deposit[:100, :, :]==40).to_sparse()
# depo2 = depo2.indices().numpy().T

# depo3 = torch.Tensor(np.logical_or( deposit[:100, :, :]==60, deposit[:100, :, :]==80)).to_sparse()
# depo3 = depo3.indices().numpy().T

substrute = torch.Tensor(deposit[20:46, 20:68, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
# point_cloud = pv.PolyData(points)
# point_cloud.plot(eye_dome_lighting=True)

p = pv.Plotter() 
p.add_mesh(depo1, color='red', point_size=10)
# p.add_mesh(depo2, color='green', point_size=5)
# p.add_mesh(depo3, color='yellow', point_size=5)
p.add_mesh(substrute, color='lightblue', point_size=10)
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(deposit[20:68, 20:46, :]==20).to_sparse()
depo1 = depo1.indices().numpy().T

# depo2 = torch.Tensor(deposit[:100, :, :]==40).to_sparse()
# depo2 = depo2.indices().numpy().T

# depo3 = torch.Tensor(np.logical_or( deposit[:100, :, :]==60, deposit[:100, :, :]==80)).to_sparse()
# depo3 = depo3.indices().numpy().T

substrute = torch.Tensor(deposit[20:68, 20:48, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
# point_cloud = pv.PolyData(points)
# point_cloud.plot(eye_dome_lighting=True)

p = pv.Plotter() 
p.add_mesh(depo1, color='red', point_size=10)
# p.add_mesh(depo2, color='green', point_size=5)
# p.add_mesh(depo3, color='yellow', point_size=5)
p.add_mesh(substrute, color='lightblue', point_size=10)
p.enable_eye_dome_lighting()
p.camera_position = 'xz'
p.camera.azimuth = 180
p.show()

In [ ]:
depo1 = torch.Tensor(deposit[:, :, :]==20).to_sparse()
depo1 = depo1.indices().numpy().T

# depo2 = torch.Tensor(deposit[:100, :, :]==40).to_sparse()
# depo2 = depo2.indices().numpy().T

# depo3 = torch.Tensor(np.logical_or( deposit[:100, :, :]==60, deposit[:100, :, :]==80)).to_sparse()
# depo3 = depo3.indices().numpy().T

substrute = torch.Tensor(deposit[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
# point_cloud = pv.PolyData(points)
# point_cloud.plot(eye_dome_lighting=True)

p = pv.Plotter() 
p.add_mesh(depo1, color='red', point_size=10)
# p.add_mesh(depo2, color='green', point_size=5)
# p.add_mesh(depo3, color='yellow', point_size=5)
p.add_mesh(substrute, color='lightblue', point_size=10)
p.enable_eye_dome_lighting()
p.show()